In [ ]:
import psycopg2

try:
    connection = psycopg2.connect(
        user="postgres",
        password="postgres",
        host="localhost",
        port="5432",
        database="pg_db"
    )
    cursor = connection.cursor()
    print("connected")
except (Exception, psycopg2.Error):
    print("Connection error")

In [ ]:
cursor.execute("""
        INSERT INTO items (item_name, category, creation_time, create_rank, rarity)
        VALUES (%s, %s, %s, %s, %s);
        """, ("Protea Blueprint", "Waframe part", "3 day", 0, "prime"))

In [106]:
import random
from datetime import datetime

cursor.execute("SELECT seller_nickname FROM sellers;")
sellers = [i[0] for i in cursor.fetchall()]
dates = [datetime(2024, 5, i, 1, 1, 1) for i in range(6, 13)]

for i in range(100):
    price = random.randint(50, 100)
    seller = sellers[random.randint(0, len(sellers) - 1)]
    date = dates[random.randint(0, len(dates) - 1)]
    cursor.execute("""
            INSERT INTO lots_history (seller, item, date_put, sale_date, price, rank)
            VALUES (%s, %s, %s, %s, %s, %s);
        """, (seller, "Protea Blueprint", datetime(2024, 5, 5, 1, 1, 1), date, price, None))

In [ ]:
import matplotlib.pyplot as plt

cursor.execute("""
        SELECT DATE(sale_date) AS date,
                AVG(price) AS average_price,
                MAX(price) AS max_price,
                MIN(price) AS min_price
        FROM lots_history
        WHERE sale_date BETWEEN '2024-05-06' AND '2024-05-13'
                AND item = 'Protea Blueprint'
        GROUP BY date
        ORDER BY date;
        """)

result = cursor.fetchall()
days = [i[0] for i in result]
avg_prices = [i[1] for i in result]
max_prices = [i[2] for i in result]
min_prices = [i[3] for i in result]

for day, max_price in zip(days, max_prices):
    plt.plot(day, max_price, marker='o', color='red', markersize=8)

for day, min_price in zip(days, min_prices):
    plt.plot(day, min_price, marker='o', color='green', markersize=8)

plt.plot(days, avg_prices, marker='o', linestyle='-')
plt.title('Средняя цена по дням')
plt.xlabel('Дата')
plt.ylabel('Цена')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
cursor.execute("""
        SELECT items_categories.category_name,
            COUNT(active_lots.id) AS number_of_lots
        FROM items_categories
            INNER JOIN items ON items_categories.category_name = items.category
            INNER JOIN active_lots ON items.item_name = active_lots.item
        GROUP BY items_categories.category_name;
        """)
result = cursor.fetchall()
categories = [i[0] for i in result]
items = [i[1] for i in result]

plt.figure(figsize=(6, 6))
plt.pie(items, labels=categories, autopct='%1.1f%%', startangle=120)
plt.title('Распределение товаров по категориям')
plt.axis('equal')
plt.tight_layout()
plt.show()


In [112]:
cursor.close()
connection.close()